In [47]:
# 必要なモジュールをインポート
import os
import json
from dotenv import load_dotenv
from openai import OpenAI
from openai.types.chat import ChatCompletionToolParam
from tavily import TavilyClient

# 環境変数の取得
load_dotenv("../.env")

# OpenAI APIクライアントを生成
client = OpenAI(api_key=os.environ['API_KEY'])

# tavily検索用APIキーの取得
TAVILY_API_KEY = os.environ['TAVILY_API_KEY']

# モデル名
MODEL_NAME = "gpt-4o-mini"

In [48]:
# 検索結果を返す関数の作成
def get_search_result(question):
    client = TavilyClient(api_key=TAVILY_API_KEY)
    response = client.search(question)
    return json.dumps({"result": response["results"]})

# テスト用コード
ret = get_search_result("東京駅のイベントを教えて")
json.loads(ret)

{'result': [{'url': 'https://www.walkerplus.com/event_list/ar0313/sc309880d/',
   'title': '東京駅(東京都)周辺のイベント - ウォーカープラス',
   'content': '開催中 2025年3月20日(祝)～6月3日(火) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都) CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ) *   [美術展・博物展](https://www.walkerplus.com/event_list/eg0107/) 開催中 2025年3月20日(祝)～6月3日(火) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 銀座一丁目駅(東京都), 東京駅(東京都) CREATIVE MUSEUM TOKYO(クリエイティブ ミュージアム トウキョウ) *   [グルメ・フードフェス](https://www.walkerplus.com/event_list/eg0117/) *   ![Image 6](https://www.walkerplus.com/asset/images/common/ico_charge.svg) 終了間近 2025年4月1日(火)～5月31日(土) 東京駅(東京都), 二重橋前駅(東京都), 京橋駅(東京都), 大手町駅(東京都), 有楽町駅(東京都) *   [グルメ・フードフェス](https://www.walkerplus.com/event_list/eg0117/) *   ![Image 8](https://www.walkerplus.com/asset/images/common/ico_parking.svg) 2025年4月5日(土)～6月15日(日) 東京駅(東京都), 二重橋前駅(東京都), 大手町駅(東京都), 日本橋駅(東京都), 京橋駅(東京都) *   [美術展・博物展](https://www.walkerplus.com/event_list/eg0107/) 2025年3月1日(土)～6月1日(日) 京橋駅(東京都), 宝町駅(東京都), 日本橋駅(東京都), 東京駅(

In [49]:
# ツール定義
def define_tools():
    return [
        ChatCompletionToolParam({
            "type": "function",
            "function": {
                "name": "get_search_result",
                "description": "指定した質問文の検索結果を取得する",
                "parameters": {
                    "type": "object",
                    "properties": {
                        "question": {"type": "string", "description": "質問文"},
                    },
                    "required": ["question"],
                },
            },
        })
    ]

In [ ]:
def make_chat_messages(system, question):
    if (system is None) or (system == ""):
        return [{"role": "user", "content": question}]
    return [
        {"role": "system", "content": f"あなたは{system}です。"},
        {"role": "user", "content": question}
    ]

In [56]:
# 言語モデルへの質問を行う関数
def ask_question(character, question, tools):
    print(f"メッセージ: {make_chat_messages(character, question)}")
    
    response = client.chat.completions.create(
        model=MODEL_NAME,
        messages=make_chat_messages(character, question),
        tools=tools,
        tool_choice="auto",
    )
    return response

In [57]:
# ツール呼び出しが必要な場合の処理を行う関数
def handle_tool_call(character, response, question):
    # 関数の実行と結果取得
    tool = response.choices[0].message.tool_calls[0]
    function_name = tool.function.name
    arguments = json.loads(tool.function.arguments)
    function_response = globals()[function_name](**arguments)

    # 関数の実行結果をmessagesに加えて再度言語モデルを呼出
    response_after_tool_call = client.chat.completions.create(
        model=MODEL_NAME,
        messages=make_chat_messages(character, question) + [
            response.choices[0].message,
            {
                "tool_call_id": tool.id,
                "role": "tool",
                "content": function_response,
            },
        ],
    )
    return response_after_tool_call

In [53]:
# ユーザーからの質問を処理する関数
def process_response(character, question, tools):
    response = ask_question(character, question, tools)

    if response.choices[0].finish_reason == 'tool_calls':
        # ツール呼出の場合
        final_response = handle_tool_call(character, response, question)
        return final_response.choices[0].message.content.strip()
    else:
        # 言語モデルが直接回答する場合
        return response.choices[0].message.content.strip()

In [54]:
tools = define_tools()

# 言語モデルが直接回答できる質問
question = "東京都と沖縄県はどちらが広いですか？"
response_message = process_response("", question, tools)
print(response_message)

東京都と沖縄県を面積で比較すると、沖縄県の方が広いです。

具体的な面積は以下の通りです：
- 沖縄県の面積は約2,281 km²です。
- 東京都の面積は約2,194 km²です。

したがって、沖縄県が東京都よりも広いということになります。


In [58]:
tools = define_tools()

# ツール呼出が必要な質問
question = "東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください"
response_message = process_response("ねこ", question, tools)
print(response_message)

メッセージ: [{'role': 'system', 'content': 'ねこ'}, {'role': 'user', 'content': '東京駅のイベントについて、最近1ヶ月以内の検索結果を教えてください'}]
最近1ヶ月以内に東京駅で開催されるイベントについて、以下の情報があります。

1. **[東京駅イベント一覧](https://bestcalendar.jp/events/%E6%9D%B1%E4%BA%AC%E9%A7%85)**
   - 開催日: 8月7日からスタートするいくつかのイベント。
   - 内容: 東京らしいドリンクを楽しめる青松酒蔵がオープンし、東京の新たなカラフルな飲食体験が提供されます。

2. **[KITTEイベントカレンダー](https://marunouchi.jp-kitte.jp/event/calendar.jsp)**
   - 期間: 2025年8月5日から8月31日までの間でイベントが行われます。
   - 内容: 様々な展示やワークショップを含むイベントが開催されます。

3. **[Walker+ - 東京のイベント情報](https://www.walkerplus.com/top/ar0313/)**
   - 内容: 多様なイベントリストが紹介されています。画期的なアート展示や食品イベントなども計画されています。

4. **[Enjoy Tokyo - 東京の楽しみ方](https://www.enjoytokyo.jp/event/list/)**
   - 様々なジャンルのイベントが続々と開催される予定で、最新情報が更新されます。

これらのイベントは、東京駅を含む東京エリアでの文化、アート、飲食など多岐にわたる楽しみ方を提供しています。詳細や日程はリンクを参照してください。


In [45]:
# チャットボットへの組み込み
tools = define_tools()

messages=[]
# システム設定
character = "気まぐれな猫"

while(True):
    # ユーザーからの質問を受付
    print("機能番号を選択してください:")
    print("1. キャラクターの設定（メッセージはクリアされます）")
    print(f"2. {character}と対話")

    question = input("機能番号を入力:")
    # 質問が入力されなければ終了
    if question.strip()=="":
        break

    elif question.strip() == "1":
        # キャラクターの設定
        character = input("キャラクターを設定してください:")
        print(f"キャラクターが設定されました: {character}")
        continue

    elif question.strip() == "2":
        question = input("メッセージを入力:")
        display(f"質問:{question}")

        # メッセージにユーザーからの質問を追加
        messages.append({"role": "user", "content": question.strip()})
        # やりとりが8を超えたら古いメッセージから削除
        if len(messages) > 8:
            # 先頭はシステム設定なので削除しない
            del_message = messages.pop(1)

        # 言語モデルに質問
        response_message = process_response(character, question, tools)

        # メッセージに言語モデルからの回答を追加
        print(response_message, flush=True)
        messages.append({"role": "assistant", "content": response_message})

    else:
        print("1か2かを入力するか、もしくはなにも入力しないでください。")
        continue

print("\n---ご利用ありがとうございました！---")

機能番号を選択してください:
1. キャラクターの設定（メッセージはクリアされます）
2. 気まぐれな猫と対話


'質問:東北6県は？'

東北地方の6県は以下の通りです：

1. 青森県
2. 岩手県
3. 宮城県
4. 秋田県
5. 山形県
6. 福島県

これらの県は日本の東北地域に位置しています。詳しく知りたい場合は、[こちらのリンク](https://ja.wikipedia.org/wiki/%E6%9D%B1%E5%8C%97%E5%9C%B0%E6%96%B9)をご覧ください。
機能番号を選択してください:
1. キャラクターの設定（メッセージはクリアされます）
2. 気まぐれな猫と対話

---ご利用ありがとうございました！---
